# The analysis of data and ideas how it can be transformed in case of predicting *SalePrice* is present in *analysis.ipynb* notebook.

In [1]:
import pandas as pd 
import numpy as np 
from scipy.stats import spearmanr
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split



c:\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Python38\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Python38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def scrollable_dataframe(df):
    
    table_html = df.to_html(classes='table',
                            table_id='scrollable_table', 
                            escape=False)

    html = f'''
    <div style="width: 100%; height:400px; overflow:scroll;">
            <style>
             td, th {{
                white-space: nowrap;
                width: 100px;
                padding: 10px;
            }}
        </style>
        {table_html}
    </div>
    '''

    return display(HTML(html))

In [3]:

data_train = pd.read_csv('train.csv')

In [24]:
X_train, X_test, y_train, y_test = train_test_split(data_train.drop(columns=['SalePrice', 'Id']), data_train[['SalePrice']], test_size=0.25, random_state=17)

## Data cleaning.
### As stated in *analysis* notebook:
##### - I will fill numerical features with median
##### - I will fill categorical features with 'Unknown' label or with majority class depending on how numerous a particular feature is
##### - I will drop columns with too numeours majority class or missing values

In [5]:
# helper function to decide how to deal with missing values for categorical columns
def unknownOrMajoritySplit(data: pd.DataFrame) -> (list, list):
    unknownCols = []
    majorityCols = []
    desc = data.describe().T
    length = len(data)
    for index, row in desc.iterrows():
        freq = row['freq']
        if freq / length >= 0.8:
            majorityCols.append(index)
        else:
            unknownCols.append(index)
    
    return (unknownCols, majorityCols)

In [6]:
cats = X_train.select_dtypes(include=[object, bool])
unknownCols, majorityCols = unknownOrMajoritySplit(cats)

In [7]:
print(unknownCols)
print(majorityCols)

['MSZoning', 'Alley', 'LotShape', 'LotConfig', 'Neighborhood', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageType', 'GarageFinish', 'PoolQC', 'Fence', 'MiscFeature']
['Street', 'LandContour', 'Utilities', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'RoofMatl', 'ExterCond', 'BsmtCond', 'BsmtFinType2', 'Heating', 'CentralAir', 'Electrical', 'Functional', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']


### The plan is following 
#### clean data -> fill_missing values separately -> preprocess with functions below

### Similar functions like below I created in *analysis.ipynb* notebook

In [8]:
import math

def adjustNumerical(data: pd.DataFrame) -> pd.DataFrame:
    copy = data.copy()
    # convert MSSubClass to categorical as it is probably not ordinal
    copy['CatMSSubClass'] = copy['MSSubClass'].astype(object)
    # convert OveralCond to categorical as it is probably not ordinal (it should be ordinal but but SalePrice does not increase as OveralQual increase)
    copy['CatOverallCond'] = copy['OverallCond'].astype(object)
    copy['CatMoSold'] = copy['MoSold'].astype(object)
    copy['CatYrSold'] = copy['YrSold'].astype(object)

    copy.drop(columns=['MoSold', 'YrSold', 'OverallCond', 'MSSubClass'])
    #copy.drop(columns='BedroomAbvGr', inplace=True)
    
    copy['TotalSqrFt'] = copy['GrLivArea'] + copy['GarageArea']
    copy['IsGarage'] = copy['GarageArea'].apply(lambda x: x if math.isnan(x) else (1 if x > 0 else 0)).astype(object)
    copy['Is2ndFlr'] = copy['2ndFlrSF'].apply(lambda x: x if math.isnan(x) else (1 if x > 0 else 0)).astype(object)

    copy.drop(columns=['1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageArea', 'TotalBsmtSF'], inplace=True)

    copy['WasRemod'] = copy['YearRemodAdd'].apply(lambda x: x if math.isnan(x) else (1 if x > 0 else 0)).astype(object)
    copy.drop(columns=['YearRemodAdd'], inplace=True)

    return copy

new_cat = ['IsGarage', 'Is2ndFlr', 'WasRemod', 'CatMSSubClass', 'CatOverallCond']
new_num = ['TotalSqrFt']
ordinal = ['OverallQual']

In [9]:
# helper functions
def qual_mapper(value):
    if value == 'Po':
        return 1
    if value == 'Fa':
        return 2
    if value == 'TA':
        return 3
    if value == 'Gd':
        return 4
    if value == 'Ex':
        return 5
    # I treat nan as 1
    return 0

def MasVnrType_mapper(value):
    if value == 'None' or value == 'BrkCmn':
        return 1
    if value == 'BrkFace':
        return 2
    if value == 'Stone':
        return 3
    # I treat nan equally as None
    return 1

def garage_mapper(value):
    if value == 'Unf':
        return 1
    if value == 'Rfn':
        return 2
    if value == 'Fin':
        return 3
    # I treat nan equally as Unf
    return 1

def toOrdinalFn(col: str):
    fn = 0
    if col == 'GarageFinish':
        fn = garage_mapper
    elif col == 'MasVnrType':
        fn = MasVnrType_mapper
    else:
        fn = qual_mapper
    return fn

In [10]:
potencial_ordinal = ["ExterQual", "FireplaceQu", "GarageFinish", "MasVnrType", "BsmtQual", "HeatingQC", "KitchenQual", "GarageQual"]

def adjustCategorical(df: pd.DataFrame) -> pd.DataFrame:
    cpy = df.copy()
    # transform some columns from categorical to ordinal
    for i in potencial_ordinal:
        cpy[i] = cpy[i].apply(toOrdinalFn(i))

    too_many_majority = ['Utilities', 'Street', 'Condition2', 'RoofMatl', 'Heating', 
                     'LandSlope', 'CentralAir', 'Functional', 'PavedDrive', 'Electrical', 'GarageCond']
    too_many_missing = ['Alley', 'PoolQC', 'MiscFeature']
    cols_to_remove = too_many_majority + too_many_missing

    cpy.drop(columns=cols_to_remove)

    return cpy

ordinal = list(set(ordinal + potencial_ordinal))

In [11]:
def adjustFeatures(df:pd.DataFrame) -> pd.DataFrame:
    cpy = df.copy()
    cpy = adjustNumerical(cpy)
    cpy = adjustCategorical(cpy)

    return cpy

In [35]:
cat_cols = list(X_train.select_dtypes(include=[object, 'category']).columns)
num_cols = list(X_train.select_dtypes(include=np.number).columns)
#num_cols.remove('SalePrice')

all_cat_cols = list(set(cat_cols + new_cat))

all_num_cols = list(set(new_num + new_num))

In [13]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

cat_unknown_transformer = Pipeline(
    steps=[('cat_unknown', SimpleImputer(missing_values='Unknown'))]
)
cat_majority_transformer = Pipeline(
    steps=[('cat_majority', SimpleImputer(strategy='most_frequent'))]
)
cat_general_transformer = Pipeline(
    steps=[('cat_general', OneHotEncoder(handle_unknown='ignore'))]
)
num_transformer = Pipeline(
    steps=[('num', SimpleImputer(strategy='median'), StandardScaler())]
)
new_features_transformer = FunctionTransformer(adjustFeatures)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat_majority', cat_majority_transformer, majorityCols),
        ('cat_unknown', cat_unknown_transformer, unknownCols),
        ('new_features', new_features_transformer, cat_cols + num_cols),
        ('cat_general_transformer', cat_general_transformer, cat_cols)
    ],remainder = 'passthrough'
)


def getPipeline(model):
    pipe = Pipeline(
    steps=[("preprocessor", preprocessor), ("model", model)]
    )

    return pipe

In [42]:
'SalePrice' in num_cols

False

In [47]:
from sklearn.linear_model import LinearRegression

cat_cols = list(X_train.select_dtypes(include=[object, 'category']).columns)
num_cols = list(X_train.select_dtypes(include=np.number).columns)

cat = Pipeline(
    steps=[('si', SimpleImputer(missing_values='Unknown', strategy='constant')), ('ohe', OneHotEncoder())]
)

num = Pipeline(
    steps=[('num', SimpleImputer())]
)

col_tr = ColumnTransformer(
    transformers=[
        ('catt', cat, cat_cols),
        ('numm', num, num_cols),
    ], remainder='passthrough'
)
#p = Pipeline(steps=[('col_tr', col_tr),('nft', new_features_transformer)])
p = Pipeline(steps=[('col_tr', col_tr), ('est', LinearRegression())])


model = p.fit(X_train, y_train)

ValueError: Input contains NaN

In [30]:
cat_unknown_transformer = Pipeline(
    steps=[('cat_unknown', SimpleImputer(strategy='constant', fill_value='Unknown'))]
)
cat_majority_transformer = Pipeline(
    steps=[('cat_majority', SimpleImputer(strategy='most_frequent'))]
)
cat_general_transformer = Pipeline(
    steps=[('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))]
)
num_transformer = Pipeline(
    steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]
)
new_features_transformer = FunctionTransformer(adjustFeatures)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat_majority', cat_majority_transformer, majorityCols),
        ('cat_unknown', cat_unknown_transformer, unknownCols),
        #('new_features', new_features_transformer, cat_cols + num_cols),
        #('cat_general_transformer', cat_general_transformer, cp)
    ], remainder='passthrough'
)

preprocessor2 = ColumnTransformer(
    transformers=[
        ('cat_general_transformer', cat_general_transformer, all_cat_cols)
    ], remainder='passthrough'
)

pipe = Pipeline(
steps=[("preprocessor", preprocessor)]
)

X_prec = pipe.fit_transform(data_train)

In [131]:
X_train_fe, X_test_fe, y_train_fe, y_test_fe = train_test_split(X_prec, data_train[['SalePrice']], test_size=0.25, random_state=42)

In [29]:
X_prec

array([[-1.730864877400689, 0.07337496353744775, -0.22087508895451458,
        ..., 'Unknown', 'Unknown', 208500],
       [-1.728492204505006, -0.8725627562389217, 0.4603197354510445, ...,
        'Unknown', 'Unknown', 181500],
       [-1.7261195316093232, 0.07337496353744775, -0.08463612407340275,
        ..., 'Unknown', 'Unknown', 223500],
       ...,
       [1.7261195316093232, 0.3098593934815401, -0.17546210066081064,
        ..., 'GdPrv', 'Shed', 266500],
       [1.728492204505006, -0.8725627562389217, -0.08463612407340275,
        ..., 'Unknown', 'Unknown', 142125],
       [1.730864877400689, -0.8725627562389217, 0.23325479398252483, ...,
        'Unknown', 'Unknown', 147500]], dtype=object)